In [1]:
!pip install psycopg2-binary pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

import json, csv
from csv import writer, reader
import os

def save2json(filename, dump):
    out_file = open(filename, "w")
    json.dump(dump, out_file, indent=6)
    out_file.close()


def save2csv(filename, header, data):
    with open(filename, "w", newline="") as csvfile:
        csvw = csv.writer(
            csvfile, delimiter=",", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        csvw.writerow(header)
        for d in data:
            csvw.writerow(d)

def delete_file(filename):
    try:
        os.remove(filename)
        print(f"File '{filename}' has been deleted.")
    except FileNotFoundError:
        print(f"File '{filename}' not found and cannot be deleted.")
    except Exception as e:
        print(f"An error occurred while deleting the file '{filename}': {str(e)}")

In [3]:
raw_data = json.load(open("/opt/airflow/notebooks/data/raw_data.json"))
df_raw_data = pd.DataFrame(raw_data)
delete_file("/opt/airflow/notebooks/data/raw_data.json")

raw_media_frames = []
metas = []
lds = []

for index, row in df_raw_data.iterrows():
    m = row.to_dict()

    if (
        "sites" in m["url"]
        or "culture" in m["url"]
        or "subculture" in m["url"]
        or "event" in m["url"]
        or "people" in m["url"]
        or "type" in m["url"]
    ):
        continue
    if (
        "content" in m
        and "about" in m["content"]
        and "text" in m["content"]["about"]
    ):
        m["content"]["about"]["fulltext"] = "".join(m["content"]["about"]["text"])
    if (
        "content" in m
        and "origin" in m["content"]
        and "text" in m["content"]["origin"]
    ):
        m["content"]["origin"]["fulltext"] = "".join(m["content"]["origin"]["text"])
    if (
        "content" in m
        and "spread" in m["content"]
        and "text" in m["content"]["spread"]
    ):
        m["content"]["spread"]["fulltext"] = "".join(m["content"]["spread"]["text"])
    if (
        "content" in m
        and "subsection" in m["content"]
        and "text" in m["content"]["subsection"]
    ):
        m["content"]["subsection"]["fulltext"] = "".join(
            m["content"]["subsection"]["text"]
        )
    if "meta" in m:
        metas.append(m.pop("meta", None))
    if "ld" in m:
        lds.append(m.pop("ld", None))

    raw_media_frames.append(m)

save2json("/opt/airflow/notebooks/data/kym.media.frames.meta.json", metas)

save2json("/opt/airflow/notebooks/data/kym.media.frames.ls.json", lds)

save2json("/opt/airflow/notebooks/data/kym.media.frames.json", raw_media_frames)

save2csv(
    "/opt/airflow/notebooks/data/kym.media.frames.csv",
    ["title", "meme"],
    [[m["title"], m["url"]] for m in raw_media_frames],
)

File '/opt/airflow/notebooks/data/raw_data.json' has been deleted.
